In [ ]:
import math
import torch
import numpy as np
import matplotlib.pyplot as plt

import pyro
import pyro.contrib.gp as gp

import torch.optim as optim

import torch.autograd as autograd

# Objective function

In [ ]:
a = -5
b = 0
c = 0.5

def test_function(X):
    return a * torch.exp(-1.0 * torch.pow((X - b), 2) / (2*c*c))

x_ = torch.linspace(-5, 5, 200)

plt.plot(x_, test_function(x_))

# Training data

In [ ]:
train_cnt = 3
X_train = torch.tensor([x for x in np.random.uniform(low=-5, high=5, size=train_cnt)])
Y_train = test_function(X_train)

# Model

In [ ]:
gpmodel = gp.models.GPRegression(X_train.T, Y_train, 
                                 gp.kernels.Matern52(input_dim=1, lengthscale=torch.ones(1), variance=torch.Tensor([150.0])), 
                                 noise=torch.tensor(0.1), 
                                 jitter=1.0e-4)

# Optimising GP's parameters

In [ ]:
optimizer = torch.optim.Adam(gpmodel.parameters(), lr=0.005)
loss_fn = pyro.infer.Trace_ELBO().differentiable_loss
losses = []
num_steps = 5000
for i in range(num_steps):
    optimizer.zero_grad()
    loss = loss_fn(gpmodel.model, gpmodel.guide)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
plt.semilogy(losses)

# Expected improvement

In [ ]:
normal_phi = lambda x: torch.exp(-x.pow(2)/2)/np.sqrt(2*np.pi)
normal_Phi = lambda x: (1 + torch.erf(x / np.sqrt(2))) / 2
  
def expected_improvement(x):
    
    y_min = gpmodel.y.min()
    
    mu, variance = gpmodel(x, full_cov=False, noiseless=False)
    
    sigma = variance.sqrt()
    
    delta = y_min - mu
    
    EI = delta.clamp_min(0.0) + sigma*normal_phi(delta/sigma) - delta.abs()*normal_Phi(delta/sigma)
    
    return -EI

# Lower confidence bound

In [ ]:
def lower_confidence_bound(x, kappa=2):
    
    mu, variance = gpmodel(x, full_cov=False, noiseless=False)
    sigma = variance.sqrt()
    
    return mu - kappa * sigma

# Plotting

In [ ]:
plt.plot(x_, expected_improvement(x_).detach().numpy(), color="red")
plt.plot(x_, lower_confidence_bound(x_).detach().numpy(), color="blue")
plt.plot(x_, test_function(x_), color="black")
plt.plot(X_train, Y_train, "*", markersize=10)

In [ ]:
X_train

# Minimising acquisition function

In [ ]:
def optimise(acquisition_func, x_st):
    
    # unconstrained minimiser
    minimizer = optim.LBFGS([x_st], lr=0.1)
                        
    def closure():
        # clear gradients
        minimizer.zero_grad()

        y = acquisition_func(x_st)

        autograd.backward(x_st, autograd.grad(y, x_st))

        print("x_st", x_st, y)

        return y

    minimizer.step(closure)

In [ ]:
x_st = torch.Tensor([0]).detach().requires_grad_(True)
optimise(expected_improvement, x_st)

In [ ]:
x_st = torch.Tensor([0.992]).detach().requires_grad_(True)
optimise(lower_confidence_bound, x_st)